# Deploying the Hybrid Test Bench Digital Twin Service

In this notebook we will deploy a service that emulates the Hybrid Test Bench DT, and subsequently create a service to store data into the time series database.

To get our service running do the followning:
1. Make sure you're in the root of our repository ([hybrid-test-bench](./))
2. Run the following command: 

```bash
    python -m startup.start_services
   ``` 

This will start both InfluxDB and RabbitMQ services. This way we can have a consistent environment accross all machines.

Local sites, usernames and passwords are the following:

- **InfluxDB:**
  - Local site: [http://localhost:8086/](http://localhost:8086/)
  - User: bench
  - Password: hybridtestbench

- **RabbitMQ:**
  - Local site: [http://localhost:15672/](http://localhost:15672/)
  - User: bench
  - Password: bench

Now you should be able to access these two services, and we'll continue from here.

## Hybrid Test Bench Digital Twin Service

The interface of the emulator is very similar to the example for the Incubator ([IncubatorDTCourse\1-Incubator-Service\1-Incubator-Service.ipynb](..\1-Incubator-Service\1-Incubator-Service.ipynb)).

1. We create the **dt_service.py**, which produces RabbitMQ messages that contain data that simulates a Digital Twin of the Hybrid Test Bench.
   
2. We create the **dt_htb_data_recorder_influx.py**, which deploys a service that subscribes to all relevant messages being exchanged, and stores them in the time series database, so that it can be visualized.

**In the end**, we check that both services are running and we check the data coming to the InfluxDB dashboard. We then send a message, that will affect the behaviour of the emulator and we will be able to see the change in the InfluxDB dashboard.

Run the next two code blocks to set up the environment and configure the logging:

In [ ]:
%%writefile logging.conf

[loggers]
keys=root,PTModel,PTEmulatorService,HybridTestBenchDataRecorderInflux,CalibrationService

[handlers]
keys=consoleHandler,PTModelFileHandler,PTEmulatorServiceFileHandler,HybridTestBenchDataRecorderInfluxFileHandler,CalibrationServiceHandler

[formatters]
keys=simpleFormatter

[logger_root]
level=DEBUG
handlers=consoleHandler

[logger_PTModel]
level=DEBUG
handlers=PTModelFileHandler
qualname=PTModel
propagate=0

[logger_PTEmulatorService]
level=DEBUG
handlers=PTEmulatorServiceFileHandler
qualname=PTEmulatorService
propagate=0

[logger_HybridTestBenchDataRecorderInflux]
level=DEBUG
handlers=HybridTestBenchDataRecorderInfluxFileHandler
qualname=HybridTestBenchDataRecorderInflux
propagate=0

[logger_CalibrationService]
level=DEBUG
handlers=CalibrationServiceHandler
qualname=CalibrationService
propagate=0

[handler_consoleHandler]
class=StreamHandler
formatter=simpleFormatter
args=(sys.stdout,)

[handler_PTModelFileHandler]
class=FileHandler
formatter=simpleFormatter
args=('PTModel.log', 'w')

[handler_PTEmulatorServiceFileHandler]
class=FileHandler
formatter=simpleFormatter
args=('PTEmulatorService.log', 'w')

[handler_HybridTestBenchDataRecorderInfluxFileHandler]
class=FileHandler
formatter=simpleFormatter
args=('HybridTestBenchDataRecorderInflux.log', 'w')

[handler_DT_HTB_DataRecorderInfluxFileHandler]
class=FileHandler
formatter=simpleFormatter
args=('DT_HTB_DataRecorderInflux.log', 'w')

[handler_CalibrationServiceHandler]
class=FileHandler
formatter=simpleFormatter
args=('CalibrationService.log', 'w')

[formatter_simpleFormatter]
format=%(asctime)s.%(msecs)03d %(levelname)s %(name)s : %(message)s
datefmt=%Y-%m-%d %H:%M:%S

Overwriting logging.conf


In [92]:
import sys
import os

# Get the current working directory. Should be hybrid-test-bench
current_dir = os.getcwd()

assert os.path.basename(current_dir) == 'hybrid-test-bench', 'Current directory is not hybrid-test-bench'

# Get the parent directory. Should be the root of the repository
parent_dir = current_dir

# The root of the repo should contain the startup folder. Otherwise something went wrong during the inital setup.
assert os.path.exists(os.path.join(parent_dir, 'startup')), 'startup folder not found in the repository root'

# The root of the repo should contain the installation folder. Otherwise something went wrong during the inital setup.
assert os.path.exists(os.path.join(parent_dir, 'installation')), 'installation folder not found in the repository root'

bench_startup_dir = os.path.join(parent_dir, 'startup')

assert os.path.exists(bench_startup_dir), 'hybrid-test-bench startup directory not found'

# Add the parent directory to sys.path
sys.path.append(bench_startup_dir)

Now that we have set up our environment, we continue by creating the DT_service script:

In [ ]:
%%writefile dt_service.py









# To-do: This needs to be rewritten! We don't need the pt_emulate part?









# Configure python path to load hybrid test bench modules
import sys
import os
import logging
import logging.config
import time
import numpy as np
from math import *

from scipy.integrate import solve_ivp

# Get the current working directory. Should be hybrid-test-bench
current_dir = os.getcwd()

assert os.path.basename(current_dir) == 'hybrid-test-bench', 'Current directory is not hybrid-test-bench'

# Get the parent directory. Should be the root of the repository
parent_dir = current_dir

from communication.server.rabbitmq import Rabbitmq
from communication.shared.protocol import ROUTING_KEY_STATE, ROUTING_KEY_FORCES
import dt_model as dt_model
import calibration_service as cal_service
import actuator_controller as actuator_controller

class DTService:
    
    def __init__(self, uh_initial, uv_initial, lh_initial, lv_initial, max_vertical_displacement, execution_interval, rabbitmq_config):
        # Initialize the DTService with initial values and configuration
        self._l = logging.getLogger("DTService")
        self._l.info("Initializing DTService.")

        self._rabbitmq = Rabbitmq(**rabbitmq_config)
        
        self.uh = uh_initial
        self.uv = uv_initial
        self.lh = lh_initial
        self.lv = lv_initial

        self.vertical_frequency = 0.0
        self.horizontal_frequency = 0.0

        self.lh_wanted = 100
        self.uv_wanted = 100

        self.max_vertical_displacement = max_vertical_displacement
        self._execution_interval = execution_interval # seconds
        self._force_on = 0.0
        self.E_modulus = 100e3 # Pa (example value for aluminum)

        try:
            self.DT_Model = dt_model.DTModel()
            self.calibration_service = cal_service.CalibrationService(self.DT_Model)
        except Exception as e:
            self._l.error("Failed to initialize DTModel: %s", e, exc_info=True)
            raise

        try:
            self.ac = actuator_controller.ActuatorController(self.lh_wanted, self.uv_wanted, self.vertical_frequency, self.horizontal_frequency, self._execution_interval)
        except Exception as e:
            self._l.error("Failed to initialize ActuatorController: %s", e, exc_info=True)
            raise

        self.DT_Model.set_beampars(16, 'E', self.E_modulus) # Set the beam parameters for the PT model  

    def setup(self):
        self._rabbitmq.connect_to_server()

        # Declare local queues for the force messages
        self.forces_queue_name = self._rabbitmq.declare_local_queue(routing_key=ROUTING_KEY_FORCES)

        self._l.info(f"DTService setup complete.")

    def _read_forces(self):
        # Read the forces from the RabbitMQ queue
        msg = self._rabbitmq.get_message(self.forces_queue_name)

        if msg is not None:
            return msg
        else:
            return None
    
    def check_control_commands(self):
        # Check if there are control commands
        force_cmd = self._read_forces()
        #self._l.debug(f"Control command: {force_cmd}")
        if force_cmd is not None:
            if 'forces' in force_cmd and force_cmd['forces'] is not None:
                self._l.info("Force command: %s", force_cmd["forces"])
                self._force_on = 1.0 if force_cmd["forces"] else 0.0

            if "horizontal_force" in force_cmd and force_cmd["horizontal_force"] is not None:
                self._l.info(f"Horizontal force command: {force_cmd['horizontal_force']}")
                self.lh_wanted = force_cmd["horizontal_force"]

            if "vertical_displacement" in force_cmd and force_cmd["vertical_displacement"] is not None:
                self._l.info(f"Vertical force command: {force_cmd['vertical_displacement']}")
                self.uv_wanted = force_cmd["vertical_displacement"]
                
            if "vertical_frequency" in force_cmd and force_cmd["vertical_frequency"] is not None:
                self._l.info(f"Vertical frequency command: {force_cmd['vertical_frequency']}")
                self.vertical_frequency = force_cmd["vertical_frequency"]
                
            if "horizontal_frequency" in force_cmd and force_cmd["horizontal_frequency"] is not None:
                self._l.info(f"Horizontal frequency command: {force_cmd['horizontal_frequency']}")
                self.horizontal_frequency = force_cmd["horizontal_frequency"]


    def emulate_pt(self):
        # Emulate the PT behavior based on the control commands

        # Additional logic for the emulator can go here
        # the if statement is just hardcoded emulator behaviour for now! 
        # _uh, _uv, _lh, _lv, and _r need to be extracted from the simulation results (u, lf, r)
        if self._force_on == 1.0:
            try:
                self._uh, self._uv, self._lh, self._lv = self.ac.step_simulation(self.PT_Model)
            except Exception as e:
                self._l.error("Failed to emulate PT behavior: %s", e, exc_info=True)
                raise

            try:
                disp = self.PT_Model.get_displacements()
                self.calibration_service.set_calibration_displacement(disp) # Set the displacements in the calibration service
                self.calibration_service.calibrate_model() # Call the calibration service to calibrate the model
            except Exception as e:
                self._l.error("Calibration service failed: %s", e, exc_info=True)
                raise
        else:
            # Horizontal displacement
            self._uh = 0.0
            # Vertical displacement
            self._uv = 0.0
            # Horizontal force
            self._lh = 0.0
            # Vertical force
            self._lv = 0.0
            # Restoring force
            # self._r = r[something] # in case we need this for the emulator, we can put it here

        self.E_modulus = self.PT_Model.get_beampars(16).E # Get the E modulus from the PT model
        self.PT_Model.set_beampars(16, 'E', self.E_modulus) # Set the E modulus in the PT model
        #self._l.info("PT script executed successfully.")
        
    def send_state(self, time_start):
        #self._l.info("Sending state to hybrid test bench physical twin.")
        timestamp = time.time_ns()
        # Publishes the new state
        message = {
            "measurement": "emulator",
            "time": timestamp,
            "tags": {
                "source": "emulator"
            },
            "fields": {
                "horizontal_displacement": self._uh,
                "vertical_displacement": self._uv,
                "horizontal_force": self._lh,
                "vertical_force": self._lv,
                "E_modulus": self.E_modulus,
                # "restoring_force": self._r,
                "force_on": self._force_on,
                "max_vertical_displacement": self.max_vertical_displacement,
                "execution_interval": self._execution_interval,
                "elapsed": time.time() - time_start,
            }
        }

        self._rabbitmq.send_message(ROUTING_KEY_STATE, message)
        #self._l.debug(f"Message sent to {ROUTING_KEY_STATE}.")
        #self._l.debug(message)
    
    def start_emulation(self):
        # Start the emulation loop
        self._l.info("Starting PTEmulator emulation loop.")
        try:
            while True:
                #self._l.debug("Emulation loop iteration.")
                time_start = time.time()
                #Check if there are control commands
                self.check_control_commands()
                # Emulate the PT behavior
                self.emulate_pt() 
                # Send the new state to the hybrid test bench physical twin
                self.send_state(time_start)
                # Sleep until the next sample
                time_end = time.time()
                time_diff = time_end - time_start
                if time_diff < self._execution_interval:
                    time.sleep(self._execution_interval - time_diff)
                else:
                    self._l.warning(f"Emulation loop took too long: {time_diff} seconds.")
        except KeyboardInterrupt:
            self._l.info("Emulation loop interrupted by user.")
        except Exception as e:
            self._l.error("Emulation loop failed: %s", e, exc_info=True)
    
if __name__ == "__main__":
    # Get utility functions to config logging and load configuration
    from pyhocon import ConfigFactory
    
    logging_conf = os.path.join(os.path.dirname(os.getcwd()), 'hybrid-test-bench', 'logging.conf')
    logging.config.fileConfig(logging_conf)

    # Get path to the startup.conf file used in the hybrid test bench PT & DT:
    startup_conf = os.path.join(os.path.dirname(os.getcwd()), 'hybrid-test-bench', 'software','startup.conf')
    assert os.path.exists(startup_conf), 'startup.conf file not found'

    # The startup.conf comes from the hybrid test bench repository.
    config = ConfigFactory.parse_file(startup_conf)
    
    service = PTEmulatorService(
        uh_initial = 0.0,
        uv_initial = 0.0,
        lh_initial = 0.0,
        lv_initial = 0.0,
        # r_initial = 0.0,
        max_vertical_displacement = 70.0,
        execution_interval = 3.0,
        rabbitmq_config=config["rabbitmq"])

    service.setup()
    
    # Start the PTEmulatorService
    service.start_emulation()

Writing dt_service.py


And we continue by creating the dt_htb_data_recorder_influx script:

The key parts of this service are:
1. It subscribes to all messages going to a topic that's prefixed by `ROUTING_KEY_RECORDER="hybridtestbench.record.#"`. So a message sent to topic `hybridtestbench.record.driver.state` will also be received by the data recorder. Note that any other service that has declared a local queue attached to that topic will receive a copy of the message.
2. It uses the InfluxDB Client library to write the message data into the time series database. The message is already in the right format.

In [2]:
%%writefile dt_htb_data_recorder_influx.py

# Configure python path to load incubator modules
import sys
import os
import logging
import logging.config
import time

from influxdb_client import InfluxDBClient
from influxdb_client.client.write_api import SYNCHRONOUS

# Get the current working directory. Should be hybrid-test-bench
current_dir = os.getcwd()

assert os.path.basename(current_dir) == 'hybrid-test-bench', 'Current directory is not hybrid-test-bench'

# Get the parent directory. Should be the root of the repository
parent_dir = current_dir

# The root of the repo should contain the startup folder. Otherwise something went wrong during the inital setup.
assert os.path.exists(os.path.join(parent_dir, 'startup')), 'startup folder not found in the repository root'

# The root of the repo should contain the installation folder. Otherwise something went wrong during the inital setup.
assert os.path.exists(os.path.join(parent_dir, 'installation')), 'installation folder not found in the repository root'

bench_startup_dir = os.path.join(parent_dir, 'startup')

assert os.path.exists(bench_startup_dir), 'hybrid-test-bench startup directory not found'

# Add the parent directory to sys.path
sys.path.append(bench_startup_dir)

from communication.shared.protocol import ROUTING_KEY_RECORDER
from communication.server.rabbitmq import Rabbitmq
import numpy as np

class DtHtbDataRecorderInflux:
    def __init__(self, rabbitmq_config, influxdb_config):
        self._l = logging.getLogger("DT_HTB_DataRecorderInflux")
        self._l.info("Initializing DT_HTB_DataRecorderInflux.")
        self._l.info("Connecting to InfluxDB...")    
        client = InfluxDBClient(**influxdb_config)
        write_api = client.write_api(write_options=SYNCHRONOUS)
        self.write_api = write_api
        self.influx_db_org = influxdb_config["org"]
        self.influxdb_bucket = influxdb_config["bucket"]

        self.rabbitmq = Rabbitmq(**rabbitmq_config)

    def read_record_request(self, ch, method, properties, body_json):
        self._l.debug("New record msg:")
        self._l.debug(body_json)
        try:
            self.write_api.write(self.influxdb_bucket, self.influx_db_org, body_json)
        except Exception as e:
            self._l.error("Failed to write to InfluxDB: %s", e, exc_info=True)
            raise

    def setup(self):
        self._l.info("Setting up DT_HTB_DataRecorderInflux.")
        self.rabbitmq.connect_to_server()

        self.rabbitmq.subscribe(routing_key=ROUTING_KEY_RECORDER,
                           on_message_callback=self.read_record_request)

    def start_recording(self):
        self._l.info("Starting DT_HTB_DataRecorderInflux.")
        try:
            self.rabbitmq.start_consuming()
        except KeyboardInterrupt:
            self.rabbitmq.close()
    
if __name__ == "__main__":
    # Get utility functions to config logging and load configuration
    from pyhocon import ConfigFactory

    # Get logging configuration
    logging.config.fileConfig("logging.conf")

    # Get path to the startup.conf file used in the hybrid test bench PT & DT:
    startup_conf = os.path.join(os.path.dirname(os.getcwd()), 'hybrid-test-bench', 'software','startup.conf')
    assert os.path.exists(startup_conf), 'startup.conf file not found'

    # The startup.conf comes from the hybrid test bench repository.
    config = ConfigFactory.parse_file(startup_conf)
    
    service = HybridTestBenchDataRecorderInflux(rabbitmq_config=config["rabbitmq"], influxdb_config=config["influxdb"])

    service.setup()
    
    # Start the HybridTestBenchDataRecorderInflux
    service.start_recording()

Writing dt_htb_data_recorder_influx.py


Now we will run both scripts.

First the DT_service:

In [ ]:
# You can run this, or you can run the command below in a new terminal windows to start the service
import subprocess
import time

# Start a process asynchronously
dt_service_proc = subprocess.Popen([sys.executable, "dt_service.py"])

# Wait for 5 seconds for the process to start
time.sleep(5)

# Print the PID of the process. You can search for this in your task manager to see the process running and kill it if necessary.
print(f"dt_service = {dt_service_proc.pid}")

pt_emulator_service = 14600


Then we run the dt_htb_data_recorder_influx:

In [3]:
# Start a process asynchronously
dt_htb_data_recorder_influx_proc = subprocess.Popen([sys.executable, "dt_htb_data_recorder_influx.py"])

# Wait for 5 seconds for the process to start
time.sleep(5)

# Print the PID of the process. You can search for this in your task manager to see the process running and kill it if necessary.
print(f"dt_htb_data_recorder_influx = {dt_htb_data_recorder_influx_proc.pid}")

NameError: name 'subprocess' is not defined

Now we check that both services are running:

In [ ]:
assert dt_service_proc.poll() is None, "DTService process has crashed"
assert dt_htb_data_recorder_influx_proc.poll() is None, "DT_HTB_DataRecorderInflux process has crashed"

We verify that the DT_service is running by printing the log file:

In [ ]:
# Print contents of log file to verify that the service is running
with open('DTService.log', 'r') as f:
    print(f.read())

# Check process hasn't crashed
assert dt_service_proc.poll() is None, "DTService process has crashed"

We verify that the dt_htb_data_recorder_influx is running by printing the log file:

In [4]:
# Print contents of log file to verify that the service is running
with open('DT_HTB_DataRecorderInflux.log', 'r') as f:
    print(f.read())

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

On the InfluxDB management page, you should be able to query for the data coming from the DT service:

To-do: Insert image here

Next, we send the 'apply force' command, adjust the displacement, and modify the frequencies:

In [5]:
from communication.server.rabbitmq import Rabbitmq
from communication.shared.protocol import ROUTING_KEY_FORCES

from pyhocon import ConfigFactory

# Get config
startup_conf = os.path.join(os.path.dirname(os.getcwd()), 'hybrid-test-bench', 'software','startup.conf')
config = ConfigFactory.parse_file(startup_conf)

# Send a message to the DT to simulate the application of a force
with Rabbitmq(**config["rabbitmq"]) as rabbitmq:
    rabbitmq.send_message(ROUTING_KEY_FORCES, {"forces": True,
                                               "horizontal_force": 500, 
                                               "vertical_displacement": 250,
                                               "horizontal_frequency": 4.0,
                                               "vertical_frequency": 2.0})

# Wait a bit for the message to be processed
time.sleep(5)

NameError: name 'os' is not defined

We can see the change in the dashboard:

To-do: Insert image here

That's it for now.

We can turn off the force application:

In [ ]:
# Get config
startup_conf = os.path.join(os.path.dirname(os.getcwd()), 'hybrid-test-bench', 'software','startup.conf')
config = ConfigFactory.parse_file(startup_conf)

# Send a message to the DT to simulate the application of a force
with Rabbitmq(**config["rabbitmq"]) as rabbitmq:
    rabbitmq.send_message(ROUTING_KEY_FORCES, {"forces": False})

# Wait a bit for the message to be processed
time.sleep(5)

In this last step we kill both processes:

In [ ]:
dt_service_proc.terminate() # Terminate the process. Send SIGTERM signal to the process. The process can catch this signal and perform cleanup operations before exiting, so we wait.
dt_service_proc.wait()  # Wait for the process to exit fully. Should print exit code 1, due to interruption.

# Ensure process has exited
assert dt_service_proc.returncode is not None, 'Process has not exited'

In [ ]:
dt_htb_data_recorder_influx_proc.terminate() # Terminate the process. Send SIGTERM signal to the process. The process can catch this signal and perform cleanup operations before exiting, so we wait.
dt_htb_data_recorder_influx_proc.wait()  # Wait for the process to exit fully. Should print exit code 1, due to interruption.

# Ensure process has exited
assert dt_htb_data_recorder_influx_proc.returncode is not None, 'Process has not exited'